In [1]:
import numpy as np
# np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [19]:
poses_arr = np.load("./datasets/nerf_llff_data/desk02/poses_bounds.npy")

In [20]:
print(poses_arr .shape)

(21, 17)


In [21]:
print(["%.3f"%x for x in poses_arr [0]])

['-0.255', '0.965', '-0.066', '-0.265', '3024.000', '0.948', '0.263', '0.179', '4.827', '4032.000', '0.190', '-0.017', '-0.982', '-0.962', '2972.516', '5.862', '13.642']


In [32]:
# test
poses = poses_arr[:, :-2].reshape([-1, 3, 5])
bds = poses_arr[:, -2:]

In [33]:
print(poses.shape)
print(bds.shape)

(21, 3, 5)
(21, 2)


In [45]:
print(poses[0])

[[  -0.255    0.965   -0.066   -0.265 3024.   ]
 [   0.948    0.263    0.179    4.827 4032.   ]
 [   0.19    -0.017   -0.982   -0.962 2972.516]]


In [37]:
print(bds[0])

[ 5.86178299 13.64211552]


In [ ]:
def create_spheric_poses(radius, n_poses=120):
    """
    Create circular poses around z axis.
    Inputs:
        radius: the (negative) height and the radius of the circle.

    Outputs:
        spheric_poses: (n_poses, 3, 4) the poses in the circular path
    """
    def spheric_pose(theta, phi, radius):
        trans_t = lambda t : np.array([
            [1,0,0,0],
            [0,1,0,-0.9*t],
            [0,0,1,t],
            [0,0,0,1],
        ])

        rot_phi = lambda phi : np.array([
            [1,0,0,0],
            [0,np.cos(phi),-np.sin(phi),0],
            [0,np.sin(phi), np.cos(phi),0],
            [0,0,0,1],
        ])

        rot_theta = lambda th : np.array([
            [np.cos(th),0,-np.sin(th),0],
            [0,1,0,0],
            [np.sin(th),0, np.cos(th),0],
            [0,0,0,1],
        ])

        c2w = rot_theta(theta) @ rot_phi(phi) @ trans_t(radius)
        c2w = np.array([[-1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]]) @ c2w
        return c2w[:3]

    spheric_poses = []
    for th in np.linspace(0, 2*np.pi, n_poses+1)[:-1]:
        spheric_poses += [spheric_pose(th, -np.pi/5, radius)] # 36 degree view downwards
    return np.stack(spheric_poses, 0)

In [2]:

def left_to_right_eye(x, y, z, theta):
    theta_a = np.arctan(y/x)
    r = np.sqrt(np.square(x) + np.square(y))
    theta_prime = 2 * np.arcsin(np.sqrt(np.square(x) + np.square(y) + np.square(z)) * np.sin(theta/2) / r)
    x_prime = r * np.cos(theta_a + theta_prime)
    y_prime = r * np.sin(theta_a + theta_prime)
    return x_prime, y_prime

In [3]:
def left_to_right_eye_sxy(x, y, z, theta):
    common_part = np.cos(theta) - np.square(z) * (1 - np.cos(theta)) / (np.square(x) + np.square(y))
    x_prime = x * common_part - y * np.sqrt(1 - np.square(common_part))
    y_prime = y * common_part + x * np.sqrt(1 - np.square(common_part))
    return x_prime, y_prime

In [4]:
print(left_to_right_eye(1,1,3,np.arcsin(0.065)))
print(left_to_right_eye_sxy(1,1,3,np.arcsin(0.065)))

(0.8362939198498173, 1.1404439835529965)
(0.8362939198498158, 1.1404439835529976)


In [7]:
print(left_to_right_eye(-0.97307063, -0.21467721, 0.45048404,np.arcsin(0.065)))
print(left_to_right_eye_sxy(-0.97307063, -0.21467721, 0.45048404,np.arcsin(0.065)))

(0.9552802278002405, 0.28353560946713846)
(-0.9552802278002405, -0.28353560946713874)


In [ ]:
[-0.97307063 -0.21467721 -0.45048404]

In [61]:
def normalize(v):
    """Normalize a vector."""
    return v/np.linalg.norm(v)

radii, focus_depth, n_poses = 1.1, 3.5, 9
poses_spiral = []
for t in np.linspace(0, 4*np.pi, n_poses+1)[:-1]: # rotate 4pi (2 rounds)
    # the parametric function of the spiral (see the interactive web)
    center = np.array([np.cos(t), -np.sin(t), -np.sin(0.5*t)]) * radii
    print("center:", center)

    # the viewing z axis is the vector pointing from the @focus_depth plane
    # to @center
    z = normalize(center - np.array([0, 0, -focus_depth]))
    print("z:", z)

    # compute other axes as in @average_poses
    y_ = np.array([0, 1, 0]) # (3)
    x = normalize(np.cross(y_, z)) # (3)
    y = np.cross(z, x) # (3)
    print("x:", x)
    print("y:", y)
    print()

    poses_spiral += [np.stack([x, y, z, center], 1)] # (3, 4)

###    Outputs:
###        poses_spiral: (n_poses, 3, 4) the poses in the spiral path
res = np.stack(poses_spiral, 0) # (n_poses, 3, 4)
print("res: ", res[0])

center: [ 1.1 -0.  -0. ]
z: [ 0.3   -0.     0.954]
x: [ 0.954  0.    -0.3  ]
y: [0. 1. 0.]

center: [ 0.191 -1.083 -0.707]
z: [ 0.064 -0.361  0.93 ]
x: [ 0.998  0.    -0.068]
y: [0.025 0.933 0.36 ]

center: [-1.034 -0.376 -1.083]
z: [-0.389 -0.142  0.91 ]
x: [ 0.919 -0.     0.393]
y: [-0.056  0.99   0.13 ]

center: [-0.55   0.953 -0.953]
z: [-0.198  0.343  0.918]
x: [ 0.977 -0.     0.211]
y: [ 0.072  0.939 -0.336]

center: [ 0.843  0.707 -0.376]
z: [0.254 0.213 0.943]
x: [ 0.965  0.    -0.26 ]
y: [-0.056  0.977 -0.206]

center: [ 0.843 -0.707  0.376]
z: [ 0.209 -0.175  0.962]
x: [ 0.977  0.    -0.212]
y: [0.037 0.984 0.171]

center: [-0.55  -0.953  0.953]
z: [-0.12  -0.208  0.971]
x: [ 0.992 -0.     0.123]
y: [-0.025  0.978  0.206]

center: [-1.034  0.376  1.083]
z: [-0.219  0.08   0.972]
x: [ 0.975 -0.     0.22 ]
y: [ 0.018  0.997 -0.078]

center: [0.191 1.083 0.707]
z: [0.044 0.249 0.967]
x: [ 0.999  0.    -0.045]
y: [-0.011  0.968 -0.249]

res:  [[ 0.954  0.     0.3    1.1  ]
 [ 0. 

In [8]:
import imageio, os
import numpy as np
# imageio.mimsave(os.path.join(dir_name, f'{args.scene_name}.gif'), imgs, fps=30)

In [9]:
dir_name = "./results/llff/"
left_eye_dir = "desk02_rotate"
right_eye_dir = "desk02_right_eye_rotate_0.045_2"
double_eye = "desk02_rotate_bino"
n_pic = 120

In [12]:
imgs = []
for i in range(n_pic):
    file_name = "%.3d.png"%i
    img_l = imageio.imread(os.path.join(dir_name, left_eye_dir, file_name))
    img_r = imageio.imread(os.path.join(dir_name, right_eye_dir, file_name))
    #新建一个拼接黑白条，可以不加
    # zero_s = np.zeros((256, 5, 3)).astype(np.uint8)
    new_pic = np.concatenate([img_l, img_r], axis=1)
    imgs.append(new_pic)

imageio.mimsave(os.path.join(dir_name, double_eye, 'fps30number120.gif'), imgs, fps=30)
imageio.mimsave(os.path.join(dir_name, double_eye, 'fps20number120.gif'), imgs, fps=20)

000
001
002
003
004
005
006
007
008
009
010
011
012
013
014
015
016
017
018
019
020
021
022
023
024
025
026
027
028
029
030
031
032
033
034
035
036
037
038
039
040
041
042
043
044
045
046
047
048
049
050
051
052
053
054
055
056
057
058
059
060
061
062
063
064
065
066
067
068
069
070
071
072
073
074
075
076
077
078
079
080
081
082
083
084
085
086
087
088
089
090
091
092
093
094
095
096
097
098
099
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
